# Assignment #2

Abhilash Vikram Gupta, Hao Lun Chu Colin, Matthew Viteri, Stephen Oluwaniyi and Yeggi Lee

In [7]:
import numpy as np
import pandas as pd
import tweepy
import GetOldTweets3 as got
import re
import textblob
import nltk

from configparser import ConfigParser
from IPython.display import display
from collections import Counter
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/jass/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

### A. Collect 5k tweets on the 2020 Presidential election. It is necessary to use general hashtags and search keywords (like 2020 U.S. election or 2020 Presidential Election) rather than “Trump” or “Biden” or “Warren”. Use Tweepy in your Python script, and also ask for the location specified by the user during registration with Twitter. 

In [9]:
def get_creds():
    config = ConfigParser()
    config.read('./credentials.ini')

    consumer_section = config['consumer']
    api_key, api_secret_key = consumer_section['api-key'], consumer_section['api-secret-key']

    access_section = config['access']
    access_token, access_token_secret = access_section['access-token'], access_section['access-token-secret']
    
    return api_key, api_secret_key, access_token, access_token_secret

try:
    consumer_key, consumer_secret, access_token, access_token_secret = get_creds()
except KeyError as e:
    raise IOError('Twitter credentials used are not included as part of the submitted assignment.' +
                 ' We have a csv with the collected data.') from e

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

OSError: Twitter credentials used are not included as part of the submitted assignment. We have a csv with the collected data.

In [10]:
display('Preparing...')
df = pd.DataFrame(columns=['Date', 'Username', 'User Location', 'Tweet'])

query = 'president election 2020'
start = '2019-01-01'
end = '2019-09-01'
count = 5000
tweetCriteria = (got.manager.TweetCriteria()
                 .setQuerySearch(query)
                 .setSince(start)
                 .setUntil(end)
                 .setMaxTweets(count))

display('Collecting tweets...')
tweets = got.manager.TweetManager.getTweets(tweetCriteria)

display('Creating dataframe...')
for tweet in tweets:
    df = df.append({
        'Date': tweet.date,
        'Username': tweet.username,
        'Tweet': tweet.text
    }, ignore_index=True)

display('Collecting user locations...')
for index, row in df.iterrows():
    if pd.isna(row['User Location']):
        df.at[index, 'User Location'] = api.get_user(row['Username']).location

display('Done.')
display(df)
df.to_csv(f'./{query}_({count})_{end}.csv', index=False)

'Preparing...'

'Collecting tweets...'

KeyboardInterrupt: 

### B.   Find FOUR key issues mentioned by the public in the tweets – e.g., character, personality, healthcare, crime, jobs, etc. (these are examples only, use wordfrequency analysis to find out). Replace words if necessary. Issues associated with Mr. Trump may not have a strong association with the democratic candidate and vice versa. So find a couple of issues that are likely to have strong association with Mr. Trump, and a couple for the democratic candidate.

In [133]:
df = pd.read_csv('president election 2020_(5000)_2019-09-01.csv', index_col=0)

In [134]:
df.head(10)

,Date,Username,User Location,Tweet
0,2019-08-31 23:46:49+00:00,MrsJaxon2004,"Maryland, USA","I’ll ask you again, can a sitting president be..."
1,2019-08-31 23:45:26+00:00,worth_two,The World,Ted Cruz is feeling uneasy about 2020 election...
2,2019-08-31 23:24:55+00:00,El_Daverino,"Texas, USA",TX Governor is not up for election until 2022....
3,2019-08-31 23:04:34+00:00,444Cin444,Looking at your star,And that is the number one reason @POTUS won’t...
4,2019-08-31 23:02:01+00:00,clevelanddotcom,"Cleveland, Ohio",Any of the top five Democratic candidates runn...
5,2019-08-31 22:49:37+00:00,nanychief,"Michigan, USA",THIS IS COMPLETE BS! WAITING UNTIL 2020 ELECTI...
6,2019-08-31 22:48:20+00:00,Aidan34655890,"East Northport, NY","I think you a great president, good luck with ..."
7,2019-08-31 22:34:04+00:00,DanielFRoche,"Cleveland, OH",Remember that it’s the President and his Cabin...
8,2019-08-31 22:32:07+00:00,ClaudiaRuff6,Follow me on Fb and Gab.ai,2020 Road To the White House the re election o...
9,2019-08-31 22:11:23+00:00,CYCYONE,"Rhode Island, USA",China called they said “We’ll wait until after...


In [135]:
same_map = {
    'trump':      ['donald', '@realdonaldtrump', 'maga', '#maga', 'trump’s', "trump's", '#trump',
                   'maga2020', '#maga2020', '#kag2020', '#kag', 'kag', 'kag2020', '#trump2020'],
    'warren':     ['elizabeth', '@ewarren', 'elizabethwarren', '@senwarren', '#elizabethwarren'],
    'biden':      ['joe', '@joebiden', "biden's"],
    'russia':     ['russian', 'putin'],
    'republican': ['gop'],
    'musk':       ['elon']
}

same_inv_map = {word: root for root, words in same_map.items() for word in words}

def splitter(data):
    words = re.sub("([^0-9A-Za-z \t]) |(\w+:\/\/\S+)", " ", data.lower()).split()
    return [textblob.Word(word).lemmatize() for word in words]

def convert_same(text):
    return [(same_inv_map[word] if word in same_inv_map else word) for word in splitter(text)]

def wordCount(data):
    filtered = [w for w in convert_same(' '.join(data['Tweet'])) if w not in stop_words]
    return Counter(filtered).most_common(400)

df['Split Tweet'] = df['Tweet'].map(convert_same)

wordCount(df)

[('president', 5221),
 ('election', 5125),
 ('2020', 5050),
 ('trump', 4461),
 ('win', 655),
 ('ha', 649),
 ('u', 586),
 ('vote', 571),
 ('democrat', 515),
 ('get', 462),
 ('presidential', 461),
 ('going', 412),
 ('would', 403),
 ('people', 377),
 ('american', 375),
 ('former', 374),
 ('america', 366),
 ('republican', 363),
 ('state', 359),
 ('voter', 338),
 ('say', 330),
 ('new', 326),
 ('like', 314),
 ('wa', 303),
 ('need', 299),
 ('want', 296),
 ('one', 289),
 ('biden', 286),
 ('2016', 285),
 ('year', 272),
 ('know', 271),
 ('day', 271),
 ('time', 270),
 ('democratic', 258),
 ('russia', 256),
 ('make', 255),
 ('mr', 251),
 ('next', 249),
 ('think', 226),
 ('country', 224),
 ('fed', 224),
 ('candidate', 202),
 ('economy', 193),
 ('via', 192),
 ('lose', 187),
 ('recession', 187),
 ('great', 185),
 ('way', 182),
 ('poll', 178),
 ('keep', 177),
 ('see', 176),
 ('run', 174),
 ('support', 174),
 ('party', 174),
 ('go', 173),
 ('ahead', 170),
 ('running', 169),
 ('good', 169),
 ('united', 

#### 4 KEY ISSUES
- economy/recession 193/187
- Russia 256
- war 91
- pro-life 92
- fake 104
- racist 84
- fraud 70
- strong 75
- hate 76
- impeachment 63

#### Same words
- donald, trump, @realdonaldtrump, maga, kag <-- make america great gain
- joe, biden, @joebiden, biden's
- elizabeth, warren, @ewarren, elizabethwarren, #elizabethwarren, @senwarren
- michael, steele
- russia, russian, putin
- president, presidents
- republicans, republican, gop
- bernie, sanders

#### NOTES
- mike, pence --> running mate of Trump 
- kag --> keep america great



In [136]:
display(df)

,Date,Username,User Location,Tweet,Split Tweet
0,2019-08-31 23:46:49+00:00,MrsJaxon2004,"Maryland, USA","I’ll ask you again, can a sitting president be...","[i’ll, ask, you, again, can, a, sitting, presi..."
1,2019-08-31 23:45:26+00:00,worth_two,The World,Ted Cruz is feeling uneasy about 2020 election...,"[ted, cruz, is, feeling, uneasy, about, 2020, ..."
2,2019-08-31 23:24:55+00:00,El_Daverino,"Texas, USA",TX Governor is not up for election until 2022....,"[tx, governor, is, not, up, for, election, unt..."
3,2019-08-31 23:04:34+00:00,444Cin444,Looking at your star,And that is the number one reason @POTUS won’t...,"[and, that, is, the, number, one, reason, @pot..."
4,2019-08-31 23:02:01+00:00,clevelanddotcom,"Cleveland, Ohio",Any of the top five Democratic candidates runn...,"[any, of, the, top, five, democratic, candidat..."
5,2019-08-31 22:49:37+00:00,nanychief,"Michigan, USA",THIS IS COMPLETE BS! WAITING UNTIL 2020 ELECTI...,"[this, is, complete, b, waiting, until, 2020, ..."
6,2019-08-31 22:48:20+00:00,Aidan34655890,"East Northport, NY","I think you a great president, good luck with ...","[i, think, you, a, great, president, good, luc..."
7,2019-08-31 22:34:04+00:00,DanielFRoche,"Cleveland, OH",Remember that it’s the President and his Cabin...,"[remember, that, it’s, the, president, and, hi..."
8,2019-08-31 22:32:07+00:00,ClaudiaRuff6,Follow me on Fb and Gab.ai,2020 Road To the White House the re election o...,"[2020, road, to, the, white, house, the, re, e..."
9,2019-08-31 22:11:23+00:00,CYCYONE,"Rhode Island, USA",China called they said “We’ll wait until after...,"[china, called, they, said, “we’ll, wait, unti..."


In [137]:
df_cleanlocation = df.dropna(subset=['User Location'])
#df_cleanlocation['User Location'] = df_cleanlocation['User Location'].map(lambda x: x if ('Michigan' in x or 'Pennsylvania' in x or 'Wisconsin' in x) else False)
#df_cleanlocation[df_cleanlocation['User Location'] != False]



In [138]:
def cleanloaction(location):
#     print(location)
    if 'Michigan' in location:
        return 'Michigan'
    if 'Pennsylvania' in location:
        return 'Pennsylvania'
    if 'Wisconsin' in location:
        return 'Wisconsin'
    return False
    
def cleandict(dataf):
    dataf['User Location'].map(lambda x: cleanloaction(x))
    return dataf

cleandict(df_cleanlocation)
i=0
data_battle = []
for x in df_cleanlocation['User Location'].map(lambda x: cleanloaction(x)):
    if x!=False:
        data_battle.append(i)
    i+=1
df_cleanlocation.iloc[data_battle]

,Date,Username,User Location,Tweet,Split Tweet
5,2019-08-31 22:49:37+00:00,nanychief,"Michigan, USA",THIS IS COMPLETE BS! WAITING UNTIL 2020 ELECTI...,"[this, is, complete, b, waiting, until, 2020, ..."
137,2019-08-31 03:35:57+00:00,ltclavinny,"Wisconsin, USA",Stopping you from seeing the wisdom of great P...,"[stopping, you, from, seeing, the, wisdom, of,..."
146,2019-08-31 02:00:20+00:00,rogerlsmith,"Grand Rapids, Michigan",The #MAGA president knows he's about to lose t...,"[the, trump, president, know, he's, about, to,..."
153,2019-08-31 00:23:41+00:00,DESciulli,"Pennsylvania, USA",Step 1: Steal 2016 election Step 2: Consolidat...,"[step, 1, steal, 2016, election, step, 2, cons..."
290,2019-08-30 10:34:26+00:00,Marine_Airwing,"Michigan, USA",Democratic contenders for the 2020 presidentia...,"[democratic, contender, for, the, 2020, presid..."
568,2019-08-29 06:01:36+00:00,mangan_mike,Michigan.,Former New York Fed president says central ban...,"[former, new, york, fed, president, say, centr..."
768,2019-08-28 17:11:34+00:00,frazac12,"Pennsylvania, USA",Trump is the best. We need him as president. 2...,"[trump, is, the, best, we, need, him, a, presi..."
1011,2019-08-28 00:45:52+00:00,nightOwlbookLvr,"Pennsylvania, USA",This upcoming 2020 Election is crucial for the...,"[this, upcoming, 2020, election, is, crucial, ..."
1024,2019-08-28 00:16:19+00:00,brct1991,"Pennsylvania, USA",My vote has officially been cast for the 2020 ...,"[my, vote, ha, officially, been, cast, for, th..."
1082,2019-08-27 21:00:39+00:00,AmericanPastors,"Pennsylvania, USA",Israel Continues to Be One of President Trump’...,"[israel, continues, to, be, one, of, president..."


In [145]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

def sentiment_analyzer_scores(sentence):
    score = analyser.polarity_scores(sentence)
    return score['compound']

sentiment_analyzer_scores("The phone is super cool.")
#The phone is super cool----------------- {'neg': 0.0, 'neu': 0.326, 'pos': 0.674, 'compound': 0.7351}

0.7351

In [147]:
df_cleanlocation['Score'] = df_cleanlocation['Tweet'].map(lambda x: sentiment_analyzer_scores(x))
df_cleanlocation

/Users/jass/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Date,Username,User Location,Tweet,Split Tweet,Score
0,2019-08-31 23:46:49+00:00,MrsJaxon2004,"Maryland, USA","I’ll ask you again, can a sitting president be...","[i’ll, ask, you, again, can, a, sitting, presi...",-0.6908
1,2019-08-31 23:45:26+00:00,worth_two,The World,Ted Cruz is feeling uneasy about 2020 election...,"[ted, cruz, is, feeling, uneasy, about, 2020, ...",-0.0772
2,2019-08-31 23:24:55+00:00,El_Daverino,"Texas, USA",TX Governor is not up for election until 2022....,"[tx, governor, is, not, up, for, election, unt...",0.4995
3,2019-08-31 23:04:34+00:00,444Cin444,Looking at your star,And that is the number one reason @POTUS won’t...,"[and, that, is, the, number, one, reason, @pot...",0.4939
4,2019-08-31 23:02:01+00:00,clevelanddotcom,"Cleveland, Ohio",Any of the top five Democratic candidates runn...,"[any, of, the, top, five, democratic, candidat...",0.2023
5,2019-08-31 22:49:37+00:00,nanychief,"Michigan, USA",THIS IS COMPLETE BS! WAITING UNTIL 2020 ELECTI...,"[this, is, complete, b, waiting, until, 2020, ...",-0.7507
6,2019-08-31 22:48:20+00:00,Aidan34655890,"East Northport, NY","I think you a great president, good luck with ...","[i, think, you, a, great, president, good, luc...",0.9535
7,2019-08-31 22:34:04+00:00,DanielFRoche,"Cleveland, OH",Remember that it’s the President and his Cabin...,"[remember, that, it’s, the, president, and, hi...",0.0000
8,2019-08-31 22:32:07+00:00,ClaudiaRuff6,Follow me on Fb and Gab.ai,2020 Road To the White House the re election o...,"[2020, road, to, the, white, house, the, re, e...",0.8934
9,2019-08-31 22:11:23+00:00,CYCYONE,"Rhode Island, USA",China called they said “We’ll wait until after...,"[china, called, they, said, “we’ll, wait, unti...",-0.6249


In [ ]:
# Calculation of lift scores
def do_lift(x, y=None):
    l1, l2 = sorted(x), sorted(y or [])
    bools = pd.DataFrame()
    for e in l1 + l2:
        bools[e] = df['words'].map(lambda words: e in words)

    def do(df, a, b):
        count_a_int_b = np.logical_and(df[a], df[b]).values.sum()
        count_a_mult_count_b = df[a].values.sum() * df[b].values.sum()
        return len(df.index) * count_a_int_b / count_a_mult_count_b

    result_df = pd.DataFrame(index=l1, columns=(l2 or l1))
    for a, b in (combinations(l1, 2) if y==None else product(l1, l2)):
        val = do(bools, a, b)
        result_df.at[a, b] = val
        if y==None:
            result_df.at[b, a] = val

    return result_df

In [130]:

for x in df_cleanlocation['User Location'].map(lambda x: x if 'mi'in x else False):
    if x !=False:
        print(x)


Miami, FL
soy soltero sin compromiso fiel amable de buen corazon y dispuesto ayudar a los demas mi cel 0979281268
Miami, FL
Miami, FL
Birmingham, AL
Middle Smithfield Township, PA
Miami, Florida, USA
Summit County, Colorado
Birmingham, AL, USA
Westminster, Ca.
Miami, FL
Surrounded by narrow minds
Miami, FL
CommieCalifornia, USA
Garramilla/Darwin NT
Wyoming, USA
Miami, FL
Miami, FL
Wyoming
I admit it, I am from Calif
commiefornia
State of illegal Immigrants 
Miami, FL
Miami, FL
miami
Hamilton, Ontario
Byron, Wyoming
Miami, FL


In [ ]:
# trump warren biden